In [9]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config, sub_root_path
from bcnf.train import Trainer
from bcnf import CondRealNVP_v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [10]:
MODEL_NAME = 'videos_double_CNN_LSTM_large'

In [11]:
config_path_pattern = os.path.join("{{BCNF_ROOT}}", "configs", "runs", f"{MODEL_NAME}.yaml")
config_path = sub_root_path(config_path_pattern)
print(f'Loading config from {config_path}')

config = load_config(config_path, verify=False)

Loading config from /home/computerman/Desktop/IGNNS/IGNNS-final-project/configs/runs/videos_double_CNN_LSTM_large.yaml


In [12]:
model = CondRealNVP_v2.from_config(config).to(device)

print(f'{model.n_params:,}')

Example camera input size: torch.Size([1, 1, 90, 160])
Output size after first layer: torch.Size([1, 5, 45, 80])
Output size after layer 1: torch.Size([1, 10, 22, 40])
Output size after layer 2: torch.Size([1, 15, 10, 19])
Final output size: 5700
ModuleList(
  (0): Sequential(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.25, inplace=False)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(10, 15, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Dropout(p=0.25, inplace=False)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
)
57,355,414


In [13]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/computerman/Desktop/IGNNS/IGNNS-final-project/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 1/1 [00:10<00:00, 10.56s/it, file=fixed_data_render_2s_15FPS_1.pkl]


Using videos data for training. Shapes:
Conditions: [torch.Size([1000, 2, 30, 90, 160]), torch.Size([1000, 7])]
Parameters: torch.Size([1000, 19])


In [14]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ichich111222333 (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -18.5715 - Val: -5.1814 (avg: -3.6735, min: -6.2362) | lr: 1.00e-04 - Patience: 200/200 - z: (0.0633 ± 0.4477) ± (1.2466 ± 0.5226):   1%|▏         | 733/50000 [1:03:49<71:29:35,  5.22s/it] 
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


distance_to_last_best_val_loss_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▂▁▂▃▁▂▃▃▄▅▅▆▇▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr_fold_-1,███████████████████████████████████▁▁▁▁▁
time_fold_-1,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_loss_fold_-1,█▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_loss_fold_-1,█▆▅▅▄▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▃▂▂▂▁▂▁▂▂▁▂▂▁▂▂▁▃▃▂▁
z_mean_mean_fold_-1,▆▂▂▃▇▄▅▃▃▄▆▅▄▅▃▄▅▃▅▅█▅▅▅▄▂▅▂▁▄▁▃▅▆▇▃▄▂▅▅
z_mean_std_fold_-1,▂▄▁▁▁▃▂▄▃▁▃▄▃▁▄▃▃▅▄▄▅▅▂▃▄▆▂▆▄▃▇▆▄▆▄▅█▅▄▄
z_std_mean_fold_-1,▂▂▂▂▁▁▂▂▄▂▅▂▃▃▃▃▄▄▃▃▇▄▄▅▃▄▄▆▅▄▇▄▃▅▆▅█▆▄▃
z_std_std_fold_-1,▃▂▂▂▂▁▃▁▇▂▄▃▃▂▃▂▂▃▂▂█▄▄▄▃▅▄▅▅▃▅▄▃▅▅▄▇▇▄▃
distance_to_last_best_val_loss_fold_-1,200


In [16]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path_pattern}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/computerman/Desktop/IGNNS/IGNNS-final-project/models/bcnf-models/videos_double_CNN_LSTM_large
